<a href="https://colab.research.google.com/github/luuloi/GWAS_Introduction_2023/blob/main/gwas_quantitative_traits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare session by install necessary packages

In [ ]:
!pip install -q rpy2
!pip install -q condacolab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 12.7 MB/s eta 0:00:00


In [ ]:
# Ignore rpy2's warnings

import warnings
from rpy2.rinterface import RRuntimeWarning

warnings.filterwarnings("ignore", category=RRuntimeWarning)

In [ ]:
# Initialize conda

import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
# Install plink

!conda install -c bioconda plink

Solving environment: - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.2



# All requested packages already installed.



# Set up R environment by installing required packages.

Load `rpy2`.

In [ ]:
%load_ext rpy2.ipython

`pacman` will make it easier to install R packages and `BiocManager` is for installing packages from the `Bioconductor` repo.

In [ ]:
%%R
if (!require("pacman")) install.packages("pacman")
library("pacman")
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

# Function to check and install missing packages
check_and_install_packages <- function(packages) {
    for (package in packages) {
        if (!require(package, character.only = TRUE)) {
            if (package %in% rownames(available.packages(repos = BiocManager::repositories())))
                BiocManager::install(package, character.only = TRUE)
        else
            install.packages(package, character.only = TRUE)
        require(package, character.only = TRUE)
        }
    }
}

NotImplementedError: ignored

We will use the following R packages for today session.

* [qqman](https://cran.r-project.org/web/packages/qqman/vignettes/qqman.html)
* [SKAT](https://cran.r-project.org/web/packages/SKAT/index.html)
* [GWASTools](https://bioconductor.org/packages/release/bioc/manuals/GWASTools/man/GWASTools.pdf)
* [gdsfmt](https://bioconductor.org/packages/release/bioc/manuals/gdsfmt/man/gdsfmt.pdf)
* [SNPRelate](https://www.bioconductor.org/packages/release/bioc/manuals/SNPRelate/man/SNPRelate.pdf)
* [GENESIS](https://bioconductor.org/packages/release/bioc/manuals/GENESIS/man/GENESIS.pdf)

In [ ]:
%%R
check_and_install_packages(c("qqman", "SKAT", "GWASTools",
                             "SNPRelate", "GENESIS", "plotly",
                             "htmlwidgets"))

# Load data

Read directly from URL

In [ ]:
%%R
# Function to read and display data
read_and_display <- function(file_path, col_names=NULL, sep=" ", header=FALSE) {
  data <- read.table(file=file_path, sep=sep, header=header, na.strings="NA")
  if (!is.null(col_names)) {
    names(data) <- col_names
  }

  cat("\n--- Head of the Data ---\n")
  print(head(data))
  cat("\n--- Dimensions of the Data ---\n")
  print(dim(data))

  return(data)
}

In [ ]:
%%R
# Read and display Transferrin individuals' info
fam_info <- read_and_display("https://faculty.washington.edu/tathornt/sisg/Transferrin.fam")

In [ ]:
%%R
# Read and display SNPs info
snp_info <- read_and_display("https://faculty.washington.edu/tathornt/sisg/Transferrin.bim", sep="\t")

In [ ]:
%%R
# Read and display Transferrin phenotype info
tpheno_info <- read_and_display("https://faculty.washington.edu/tathornt/sisg/Tr.pheno", col_names=c("FAMID", "ID", "Transferrin"))
cat("\n--- Summary of Transferrin ---\n")
print(summary(tpheno_info[["Transferrin"]]))
cat("\n--- NA counts in Transferrin ---\n")
print(table(is.na(tpheno_info[["Transferrin"]])))
hist(tpheno_info[["Transferrin"]], xlab="Transferrin", main="Histogram of Transferrin")

In [ ]:
%%R
# Read and display Height phenotype info
hpheno_info <- read_and_display("https://faculty.washington.edu/tathornt/sisg/Ht.pheno", col_names=c("FAMID", "ID", "Height"))
cat("\n--- Summary of Height ---\n")
print(summary(hpheno_info[["Height"]]))
cat("\n--- NA counts in Height ---\n")
print(table(is.na(hpheno_info[["Height"]])))
hist(hpheno_info[["Height"]], xlab="Height", main="Histogram of Height")

In [ ]:
%%bash

## Download required data files
# Check and download Transferrin.bim if not present
[ ! -f Transferrin.bim ] && wget https://faculty.washington.edu/tathornt/sisg/Transferrin.bim -O Transferrin.bim

# Check and download Transferrin.fam if not present
[ ! -f Transferrin.fam ] && wget https://faculty.washington.edu/tathornt/sisg/Transferrin.fam -O Transferrin.fam

# Check and download Transferrin.bed if not present
[ ! -f Transferrin.bed ] && wget https://dl.dropboxusercontent.com/s/1ub8yc7jwpggm4s/Transferrin.bed

# Check and download Tr.pheno if not present
[ ! -f Tr.pheno ] && wget https://faculty.washington.edu/tathornt/sisg/Tr.pheno -O Tr.pheno

# Check and download Ht.pheno if not present
[ ! -f Ht.pheno ] && wget https://faculty.washington.edu/tathornt/sisg/Ht.pheno -O Ht.pheno

## Association analysis with Transferrin phenotype
plink --bfile Transferrin --pheno Tr.pheno --maf 0.05 --geno 0.01 --hwe 0.001 --assoc --out GWAS_T_add

## Association analysis with Height phenotype ###
plink --bfile Transferrin --pheno Ht.pheno --maf 0.05 --geno 0.01 --hwe 0.001 --assoc --out GWAS_H_add

The warnings received are indicative of common challenges when conducting quantitative trait (QT) association tests with PLINK.

* **Quantitative Trait** with `--assoc`:
The default `--assoc` in PLINK is for case/control phenotypes. When the phenotype is quantitative (like height, weight, etc.), we should use `--linear` for linear regression-based association.

* **X Chromosome** and `--hwe`:
The Hardy-Weinberg equilibrium (HWE) testing in PLINK can produce slightly misleading results for the X chromosome due to its nature (males have only one copy, females have two). As the warning suggests, we might consider using a less stringent HWE threshold for the X chromosome or exclude X chromosome variants from HWE testing using `--not-chr X`.

In [ ]:
%%bash

## Association analysis using adjusted params with Transferrin phenotype
plink --bfile Transferrin --pheno Tr.pheno --maf 0.05 --geno 0.01 --hwe 0.001 --linear --out GWAS_T_add --not-chr X

## Association analysis using adjusted params with Height phenotype ###
plink --bfile Transferrin --pheno Ht.pheno --maf 0.05 --geno 0.01 --hwe 0.001 --linear --out GWAS_H_add --not-chr X

In [ ]:
%%R
# Load the GWASTools package so that we can use the plotting functions for this package
library(GWASTools)

# Read in the Association Results from PLINK for Height
Height.Assoc = read.table("GWAS_H_add.qassoc", header=TRUE)
head(Height.Assoc)

# Obtain a Manhattan plot for Height
manhattanPlot(p=Height.Assoc[["P"]],
              chromosome=Height.Assoc[["CHR"]],
              thinThreshold=1e-4,
              main="Association Results for Height")

# Obtain a Q-Q plot for Height
qqPlot(pval=Height.Assoc[["P"]], thinThreshold=1e-4, main="Association Results for Height")

# Obtain Manhattan plot for just the Height-related autosomes
AHeight.Assoc <- subset(Height.Assoc, CHR <= 22)
manhattanPlot(p=AHeight.Assoc[["P"]],
              chromosome=AHeight.Assoc[["CHR"]],
              thinThreshold=1e-4,
              main="Association Results for Autosomes for Height")

# Obtain a Q-Q plot for Height-related autosomes
qqPlot(pval=AHeight.Assoc[["P"]],
       thinThreshold=1e-4,
       main="Association Results of Autosomes for Height")

In [ ]:
%%R
# Alternatively, we can use the qqman package to obtain a Manhattan plot
library("qqman")

manhattan(Height.Assoc)
qq(Height.Assoc[["P"]])

# Identify top 10 SNPs for Height
head(Height.Assoc)
dim(Height.Assoc) # number of SNPs analyzed

TOP <- Height.Assoc[order(Height.Assoc[["P"]]),]
head(TOP, 10, )

In [ ]:
%%R

install.packages("plotly")

In [ ]:
%%R

## Interactive Manhattan plot
# Load plotly
library(plotly)
library(htmlwidgets)

# Sample data (replace this with your data)
Height.Assoc <- read.table("GWAS_H_add.qassoc", header=TRUE)

# Create a color factor based on chromosomes
Height.Assoc$color <- as.factor(Height.Assoc$CHR)

# Create interactive plot
p <- plot_ly(Height.Assoc, x = ~BP, y = ~-log10(P), text = ~SNP,
             color = ~color, colors = "Set1",
             type = 'scatter', mode = 'markers',
             marker = list(size = 5, opacity = 0.5)) %>%
    layout(title = "Manhattan Plot",
           xaxis = list(title = "Position"),
           yaxis = list(title = "-log10(p)"),
           hovermode = "closest")

# Save the plot as an HTML file
saveWidget(p, "plotly.html", selfcontained = F)

In [ ]:
from IPython.display import display, HTML

with open('plotly.html', 'r') as f:
    display(HTML(f.read()))

In [ ]:
## Interactive Manhattan plot
import pandas as pd
import plotly.express as px
import numpy as np

# Sample data (replace this with your data)
data = pd.read_csv("GWAS_H_add.qassoc", sep="\s+", header=0)

# Create a new column for color
data['color'] = data['CHR'].astype(str)

# Separate the points in 'bins' on the x-axis by adjusting their positions
spacing = 5e6  # Space between chromosomes
previous_max = 0

chromosome_midpoints = []

for chr_num in sorted(df['CHR'].unique()):
    max_chr_position = df[df['CHR'] == chr_num]['BP'].max()
    midpoint = previous_max + (max_chr_position / 2)
    chromosome_midpoints.append(midpoint)

    df.loc[df['CHR'] == chr_num, 'adjusted_BP'] = df['BP'] + previous_max
    previous_max += max_chr_position + spacing

# Define the color scale
color_scale = px.colors.qualitative.Set1

# Create the Manhattan plot
fig = px.scatter(data, x='BP', y=-np.log10(data['P']), color='color',
                 color_discrete_sequence=color_scale,
                 hover_data=["SNP"], title="Manhattan Plot")

# Set threshold at a p-value of 5e−8
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=0, x1=df['BP'].max(), y0=7.3, y1=7.3
)

# Update y-axis label
fig.update_layout(yaxis_title="-log10(p)")

# Show plot
fig.show()